In [36]:
import pandas as pd
import calendar
import hvplot.pandas
from sqlalchemy import create_engine
import csv
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime as dt
from datetime import datetime, timedelta


warnings.filterwarnings('ignore')

In [37]:
# import house csv data
senator_csv = ('senate_stock_csv.csv')

# read csv with pandas
senator_df = pd.read_csv(senator_csv).dropna()

# view the df
print(senator_df)

     transaction_date   owner ticker asset_type            type     amount  \
0           4/18/2023  Spouse    ESS      Stock     Sale (Full)   $15,000    
1           4/18/2023    Self    ESS      Stock     Sale (Full)   $15,000    
3            4/4/2023  Spouse    UPS      Stock     Sale (Full)   $15,000    
4            4/4/2023  Spouse    MCD      Stock  Sale (Partial)   $15,000    
5            4/4/2023  Spouse    DUK      Stock     Sale (Full)   $50,000    
...               ...     ...    ...        ...             ...        ...   
7627        1/16/2015    Self    NOG      Stock        Purchase   $50,000    
7628         1/6/2015    Self    FCX      Stock     Sale (Full)  $100,000    
7640       12/12/2014  Spouse    XOM      Stock     Sale (Full)   $15,000    
7641       12/19/2014  Spouse     BA      Stock        Purchase   $15,000    
7642       12/12/2014  Spouse    BAC      Stock     Sale (Full)   $15,000    

           party state                           industry  \
0 

In [38]:
# separate senators by party and amount
keep_columns = ['party', 'amount']

# initiate variable with keep_columns
senator_df = senator_df[keep_columns]

# view resutls
print(senator_party)

           party                  name    amount ticker
0       Democrat    Sheldon Whitehouse  $15,000     ESS
1       Democrat    Sheldon Whitehouse  $15,000     ESS
3     Republican  Shelley Moore Capito  $15,000     UPS
4     Republican  Shelley Moore Capito  $15,000     MCD
5     Republican  Shelley Moore Capito  $50,000     DUK
...          ...                   ...       ...    ...
8210  Republican      Susan M. Collins  $15,000       C
8211  Republican      Susan M. Collins  $15,000    EBAY
8212  Republican      Susan M. Collins  $15,000      FB
8213  Republican      Susan M. Collins  $15,000     CRM
8214  Republican      Susan M. Collins  $15,000      GE

[5897 rows x 4 columns]


In [39]:
democrat_party = senator_df[senator_df['party'] == 'Democrat']
republican_party = senator_df[senator_df['party'] == 'Republican']

print("Democrat DataFrame:")
print(democrat_party)

print("\nRepublican DataFrame:")
print(republican_party)


Democrat DataFrame:
         party    amount
0     Democrat  $15,000 
1     Democrat  $15,000 
6     Democrat  $50,000 
7     Democrat  $15,000 
8     Democrat  $50,000 
...        ...       ...
7610  Democrat  $15,000 
7612  Democrat  $50,000 
7613  Democrat  $50,000 
7615  Democrat  $50,000 
7616  Democrat  $15,000 

[1340 rows x 2 columns]

Republican DataFrame:
           party     amount
3     Republican   $15,000 
4     Republican   $15,000 
5     Republican   $50,000 
19    Republican   $15,000 
20    Republican   $15,000 
...          ...        ...
7627  Republican   $50,000 
7628  Republican  $100,000 
7640  Republican   $15,000 
7641  Republican   $15,000 
7642  Republican   $15,000 

[4134 rows x 2 columns]


In [40]:
democrat_party = democrat_party[keep_columns]
print(democrat_party)

democrat_party['amount'] = democrat_party['amount'].replace({'\$': '', ',': ''}, regex=True).astype(float)

dparty_total_sum_amount = democrat_party['amount'].sum()
print(dparty_total_sum_amount)

         party    amount
0     Democrat  $15,000 
1     Democrat  $15,000 
6     Democrat  $50,000 
7     Democrat  $15,000 
8     Democrat  $50,000 
...        ...       ...
7610  Democrat  $15,000 
7612  Democrat  $50,000 
7613  Democrat  $50,000 
7615  Democrat  $50,000 
7616  Democrat  $15,000 

[1340 rows x 2 columns]
51985000.0


In [41]:
republican_party = republican_party[keep_columns]
print(republican_party)


republican_party['amount'] = republican_party['amount'].replace({'\$': '', ',': ''}, regex=True).astype(float)

rparty_total_sum_amount = republican_party['amount'].sum()
display(rparty_total_sum_amount)


           party     amount
3     Republican   $15,000 
4     Republican   $15,000 
5     Republican   $50,000 
19    Republican   $15,000 
20    Republican   $15,000 
...          ...        ...
7627  Republican   $50,000 
7628  Republican  $100,000 
7640  Republican   $15,000 
7641  Republican   $15,000 
7642  Republican   $15,000 

[4134 rows x 2 columns]


388695000.0

In [42]:
party_combine = pd.DataFrame({
    'party': ['Democrat', 'Republican'],
    'total_sum_amount': [dparty_total_sum_amount, rparty_total_sum_amount]
})





# use hvplot to create a pie chart
party_colors = {'Democrat': 'blue', 'Republican': 'red'}
bar_chart = party_combine.hvplot.bar(
    x='party',
    y='total_sum_amount',
    ylabel="Total Trades Amounts In Millions",
    yformatter="%0.0f",
    color='party',
    cmap = party_colors,  # Specify custom colors for each bar
    rot=0,
    title='Senator Trade Totals By Party'
)

# Show the chart using hvplot
bar_chart

:Bars   [party]   (total_sum_amount)

In [ ]:
# Next Compare Ticker Data

In [43]:
# read in senator csv
senator_csv = ('senate_stock_csv.csv')
senator_df = pd.read_csv(senator_csv).dropna()
# preview senator dataframe
senator_df

,transaction_date,owner,ticker,asset_type,type,amount,party,state,industry,senator
0,4/18/2023,Spouse,ESS,Stock,Sale (Full),"$15,000",Democrat,RI,Real Estate Investment Trusts,Sheldon Whitehouse
1,4/18/2023,Self,ESS,Stock,Sale (Full),"$15,000",Democrat,RI,Real Estate Investment Trusts,Sheldon Whitehouse
3,4/4/2023,Spouse,UPS,Stock,Sale (Full),"$15,000",Republican,WV,Trucking Freight/Courier Services,Shelley Moore Capito
4,4/4/2023,Spouse,MCD,Stock,Sale (Partial),"$15,000",Republican,WV,Restaurants,Shelley Moore Capito
5,4/4/2023,Spouse,DUK,Stock,Sale (Full),"$50,000",Republican,WV,Power Generation,Shelley Moore Capito
...,...,...,...,...,...,...,...,...,...,...
7627,1/16/2015,Self,NOG,Stock,Purchase,"$50,000",Republican,ND,Oil & Gas Production,John Hoeven
7628,1/6/2015,Self,FCX,Stock,Sale (Full),"$100,000",Republican,ND,Precious Metals,John Hoeven
7640,12/12/2014,Spouse,XOM,Stock,Sale (Full),"$15,000",Republican,KS,Integrated oil Companies,Pat Roberts
7641,12/19/2014,Spouse,BA,Stock,Purchase,"$15,000",Republican,KS,Aerospace,Pat Roberts


In [44]:
# modify dataframe to contain desired values
ticker_df = senator_df.drop(columns=['transaction_date','owner','asset_type','type','amount','state','senator'])
top_five = ['Major Banks',
            'Major Pharmaceuticals',
            'Semiconductors',
            'Computer Software: Prepackaged Software',
            'Computer Manufacturing'
           ]
# get industry top 5
ticker_df = ticker_df.loc[ticker_df['industry'].isin(top_five)].reset_index().drop(columns='index')
# find the tallys
ticker_df = ticker_df.value_counts()
# view the df
ticker_df

ticker  party       industry                               
AAPL    Republican  Computer Manufacturing                     125
MSFT    Republican  Computer Software: Prepackaged Software     80
BAC     Republican  Major Banks                                 75
PFE     Republican  Major Pharmaceuticals                       71
WFC     Republican  Major Banks                                 47
                                                              ... 
SHOP    Republican  Computer Software: Prepackaged Software      1
SMTC    Republican  Semiconductors                               1
SNOW    Democrat    Computer Software: Prepackaged Software      1
SNPS    Democrat    Computer Software: Prepackaged Software      1
ALGM    Democrat    Semiconductors                               1
Name: count, Length: 155, dtype: int64

In [45]:
# plot the data as a bar chart by industry
ticker_df.hvplot.bar(
    title="Ticker Count by Industry",
    x='ticker',
    groupby='industry',
    width=1200,
    height=400,
    xlabel='Ticker',
    ylabel='Count',
    widget_location='right',
    rot=30
)

Row
    [0] HoloViews(DynamicMap, height=400, sizing_mode='fixed', widget_location='right', width=1200)
    [1] WidgetBox(align=('end', 'center'))
        [0] Select(margin=(20, 20, 20, 20), name='industry', options=['Computer Manufacturing',...], value='Computer Manufacturing', width=250)

In [46]:
#Regrab data to categorize by senator
ticker_senator_df = senator_df.drop(columns=['transaction_date','owner','asset_type','type','amount','state','industry'])
top_five = ['David Perdue',
            'Tommy Tuberville',
            'Sheldon Whitehouse',
            'Pat Roberts',
            'Shelley Moore Capito'
           ]
ticker_senator_df = ticker_senator_df.loc[ticker_senator_df['senator'].isin(top_five)].reset_index().drop(columns='index')
ticker_senator_df = ticker_senator_df.value_counts()
ticker_senator_df

ticker  party       senator             
DISCA   Republican  David Perdue            74
URBN    Republican  David Perdue            65
CZR     Republican  David Perdue            63
PFE     Republican  David Perdue            62
FEYE    Republican  David Perdue            60
                                            ..
A       Republican  Tommy Tuberville         1
JKHY    Republican  Shelley Moore Capito     1
JCI     Republican  David Perdue             1
ITW     Republican  Tommy Tuberville         1
ZTS     Republican  Tommy Tuberville         1
Name: count, Length: 644, dtype: int64

In [47]:
# plot the data as a bar chart by senator
ticker_senator_df.hvplot.bar(
    title="Ticker Count by Senator",
    x='ticker',
    groupby='senator',
    width=1200,
    height=400,
    xlabel='Ticker',
    ylabel='Count',
    widget_location='right',
    rot=30
)

Row
    [0] HoloViews(DynamicMap, height=400, sizing_mode='fixed', widget_location='right', width=1200)
    [1] WidgetBox(align=('end', 'center'))
        [0] Select(margin=(20, 20, 20, 20), name='senator', options=['David Perdue', ...], value='David Perdue', width=250)

In [76]:
# Separate By State


In [77]:
csv_path = 'cleaned_senators_data.csv'


In [82]:
# Read in the CSV as a DataFrame
senators_df = pd.read_csv(csv_path)
senators_df.head()

,state,senator,Unnamed: 2,zip
0,RI,Sheldon Whitehouse,NaN,2903.0
1,RI,Sheldon Whitehouse,NaN,2903.0
2,WV,Shelley Moore Capito,NaN,25305.0
3,WV,Shelley Moore Capito,NaN,25305.0
4,WV,Shelley Moore Capito,NaN,25305.0


In [83]:
# Sort ascending (default)
sort_senators = senators_df.sort_values("state")
sort_senators

,state,senator,Unnamed: 2,zip
3671,AK,Dan Sullivan,NaN,NaN
304,AK,Dan Sullivan,NaN,99801.0
303,AK,Dan Sullivan,NaN,99801.0
302,AK,Dan Sullivan,NaN,99801.0
301,AK,Dan Sullivan,NaN,99801.0
...,...,...,...,...
4209,WV,Shelley Moore Capito,NaN,NaN
3325,WV,Shelley Moore Capito,NaN,NaN
1015,WY,Cynthia M. Lummis,NaN,NaN
1014,WY,Cynthia M. Lummis,NaN,NaN


In [84]:
sort_senators

,state,senator,Unnamed: 2,zip
3671,AK,Dan Sullivan,NaN,NaN
304,AK,Dan Sullivan,NaN,99801.0
303,AK,Dan Sullivan,NaN,99801.0
302,AK,Dan Sullivan,NaN,99801.0
301,AK,Dan Sullivan,NaN,99801.0
...,...,...,...,...
4209,WV,Shelley Moore Capito,NaN,NaN
3325,WV,Shelley Moore Capito,NaN,NaN
1015,WY,Cynthia M. Lummis,NaN,NaN
1014,WY,Cynthia M. Lummis,NaN,NaN


In [85]:
# Using the read_csv function and Path module, create a DataFrame 
# by importing the sfo_neighborhoods_census_data.csv file from the Resources folder
#importing and parsing census daata by year
senators_df = pd.read_csv(('cleaned_senators_sort_data.csv'), parse_dates=False, infer_datetime_format=True, index_col='state')

# Review the first and last five rows of the DataFrame
senators_df.head()

C:\Users\Name\AppData\Local\Temp\ipykernel_22836\2982263450.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  senators_df = pd.read_csv(('cleaned_senators_sort_data.csv'), parse_dates=False, infer_datetime_format=True, index_col='state')


,senator,zip
state,,
AK,Dan Sullivan,99801
AK,Dan Sullivan,99801
AK,Dan Sullivan,99801
AK,Dan Sullivan,99801
AK,Dan Sullivan,99801


In [86]:
#groupby state
combine_trans = senators_df.groupby('state').count()

#drop zips
combine_trans.drop(columns=['zip'], inplace=True)
combine_trans

,senator
state,
AK,82
AL,676
AR,1
CO,53
DE,651
GA,2553
IL,23
KS,589
KY,22


In [87]:
# group by 'state'
combine_trans = senators_df.groupby('state').count()

# drop zip column
combine_trans.drop(columns=['zip'], inplace=True)
combine_trans

,senator
state,
AK,82
AL,676
AR,1
CO,53
DE,651
GA,2553
IL,23
KS,589
KY,22


In [88]:
# get a count of senators trades by state
sen_by_state = senators_df.groupby('senator').count()
sen_by_state

,zip
senator,
"Angus S. King, Jr.",34
Benjamin L Cardin,3
Bill Cassidy,164
Bill Hagerty,27
Chris Van Hollen,7
Christopher A. Coons,7
Cory A Booker,13
Cynthia M. Lummis,2
Dan Sullivan,82


In [89]:
#plot by state
sen_by_state.hvplot(kind='bar', title='Transactions per Senator', 
                                        width=1000,
                                        height=600,
                                         xlabel='States', 
                                         ylabel='Transactions',
                                         colormap='grey',
                                         hover_color='yellow')

:Bars   [senator]   (zip)

In [91]:
# Use the hvplot function to plot the transactions per State as a bar chart. The x-axis represent the Transactions and the y-axis represent the States.¶

combine_trans.hvplot(kind='bar', title='Senators Transactions per State',
                                         width=1000,
                                         height=600,
                                         xlabel='States', 
                                         ylabel='Transactions',
                                         colormap='grey',
                                         hover_color='yellow')

:Bars   [state]   (senator)

In [93]:
# Load neighborhoods coordinates data and set index_col to Neighborhood
state_locations_df = pd.read_csv('senators_coordinates.csv')

# Review the DataFrame
state_locations_df

,state,senator,Lat,Lon
0,AK,Dan Sullivan,63.5888,-154.4931
1,AK,Dan Sullivan,63.5888,-154.4931
2,AK,Dan Sullivan,63.5888,-154.4931
3,AK,Dan Sullivan,63.5888,-154.4931
4,AK,Dan Sullivan,63.5888,-154.4931
...,...,...,...,...
7637,WV,Shelley Moore Capito,38.5976,-80.4549
7638,WV,Shelley Moore Capito,38.5976,-80.4549
7639,WY,Cynthia M. Lummis,43.0760,-107.2903
7640,WY,Cynthia M. Lummis,43.0760,-107.2903


In [94]:
# Create a DataFrame consisting of the senator, Latitude and Longitude columns
senator_plot_df = state_locations_df[["state", "senator", "Lat", "Lon"]]

# Review the DataFrame
senator_plot_df.head()

,state,senator,Lat,Lon
0,AK,Dan Sullivan,63.5888,-154.4931
1,AK,Dan Sullivan,63.5888,-154.4931
2,AK,Dan Sullivan,63.5888,-154.4931
3,AK,Dan Sullivan,63.5888,-154.4931
4,AK,Dan Sullivan,63.5888,-154.4931


In [95]:
state_locations_df.groupby('state').senator.count()

state
AK      82
AL     676
AR       1
CO      53
DE     651
GA    2553
IL      23
KS     589
KY      22
LA     165
MD      10
ME     383
MI      54
MN      24
MO       4
MS       4
NC      97
ND     184
NE       4
NJ      13
NM       2
NV      10
OK     217
OR     223
PA     149
RI     844
SC       3
SD       2
TN      27
TX      10
VA      10
WA     167
WV     383
WY       3
Name: senator, dtype: int64

In [168]:
# Create a plot to analyze Senator info

map_plot = state_locations_df.hvplot.points(
    'Lon', 
    'Lat', 
    geo=True,
    color='senator',
    scale = .04,
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 600
    )

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (senator)

In [98]:
# Using the read_csv function and Path module, create a DataFrame 
# by importing the sfo_neighborhoods_census_data.csv file from the Resources folder
#importing and parsing census daata by year
industry_df = pd.read_csv(('cleaned_senators_industry.csv'), parse_dates=False, infer_datetime_format=True)

# Review the first and last five rows of the DataFrame
industry_df.head()

C:\Users\Name\AppData\Local\Temp\ipykernel_22836\3871593057.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  industry_df = pd.read_csv(('cleaned_senators_industry.csv'), parse_dates=False, infer_datetime_format=True)


,industry,senator
0,Real Estate Investment Trusts,Sheldon Whitehouse
1,Real Estate Investment Trusts,Sheldon Whitehouse
2,Trucking Freight/Courier Services,Shelley Moore Capito
3,Restaurants,Shelley Moore Capito
4,Power Generation,Shelley Moore Capito


In [99]:
# Create a DataFrame consisting of the senator, Latitude and Longitude columns
industry_plot_df = industry_df[["industry", "senator"]]

# Review the DataFrame
industry_plot_df.head()

,industry,senator
0,Real Estate Investment Trusts,Sheldon Whitehouse
1,Real Estate Investment Trusts,Sheldon Whitehouse
2,Trucking Freight/Courier Services,Shelley Moore Capito
3,Restaurants,Shelley Moore Capito
4,Power Generation,Shelley Moore Capito


In [100]:
industry_plot_df['industry'].value_counts()

industry
Major Banks                                              344
Major Pharmaceuticals                                    282
Semiconductors                                           221
Computer Software: Prepackaged Software                  219
Computer Manufacturing                                   218
                                                        ... 
Trusts, Except Educational, Religious, and Charitable      1
 Medicinal Chemicals and Botanical Products                1
Tools/Hardware                                             1
Managed Health Care                                        1
Security Systems Services                                  1
Name: count, Length: 122, dtype: int64

In [ ]:
import hvplot.pandas as hvplot
%matplotlib inline

# Compare tickers using Yfinance Library

In [102]:
# initiate csv variable
tickers_data = "senate_stock_csv.csv"

# use pandas to read csv
tickers_data_df = pd.read_csv(tickers_data, infer_datetime_format=True, parse_dates=True)

tickers_data_df

C:\Users\Name\AppData\Local\Temp\ipykernel_22836\3710887739.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  tickers_data_df = pd.read_csv(tickers_data, infer_datetime_format=True, parse_dates=True)


,transaction_date,owner,ticker,asset_type,type,amount,party,state,industry,senator
0,4/18/2023,Spouse,ESS,Stock,Sale (Full),"$15,000",Democrat,RI,Real Estate Investment Trusts,Sheldon Whitehouse
1,4/18/2023,Self,ESS,Stock,Sale (Full),"$15,000",Democrat,RI,Real Estate Investment Trusts,Sheldon Whitehouse
2,5/16/2023,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,NaN,Michael F. Bennet
3,4/4/2023,Spouse,UPS,Stock,Sale (Full),"$15,000",Republican,WV,Trucking Freight/Courier Services,Shelley Moore Capito
4,4/4/2023,Spouse,MCD,Stock,Sale (Partial),"$15,000",Republican,WV,Restaurants,Shelley Moore Capito
...,...,...,...,...,...,...,...,...,...,...
8441,8/17/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,NaN,Sheldon Whitehouse
8442,8/16/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,NaN,Pat Roberts
8443,8/15/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,NaN,Rob Portman
8444,8/2/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,NaN,Thomas R. Carper


In [103]:
# dropna from tickers
tickers_df_dropped=tickers_data_df.dropna()
tickers_df_dropped.head

<bound method NDFrame.head of      transaction_date   owner ticker asset_type            type     amount  \
0           4/18/2023  Spouse    ESS      Stock     Sale (Full)   $15,000    
1           4/18/2023    Self    ESS      Stock     Sale (Full)   $15,000    
3            4/4/2023  Spouse    UPS      Stock     Sale (Full)   $15,000    
4            4/4/2023  Spouse    MCD      Stock  Sale (Partial)   $15,000    
5            4/4/2023  Spouse    DUK      Stock     Sale (Full)   $50,000    
...               ...     ...    ...        ...             ...        ...   
7627        1/16/2015    Self    NOG      Stock        Purchase   $50,000    
7628         1/6/2015    Self    FCX      Stock     Sale (Full)  $100,000    
7640       12/12/2014  Spouse    XOM      Stock     Sale (Full)   $15,000    
7641       12/19/2014  Spouse     BA      Stock        Purchase   $15,000    
7642       12/12/2014  Spouse    BAC      Stock     Sale (Full)   $15,000    

           party state           

In [104]:
# create dataframe with tickers_df
tickers_df=pd.DataFrame(tickers_df_dropped)
# tickers_df

# display the transaction_date, ticker, type, amount and senator columns
tickers_df=tickers_df[['transaction_date', 'ticker', 'type', 'amount', 'senator']]
display(tickers_df)

,transaction_date,ticker,type,amount,senator
0,4/18/2023,ESS,Sale (Full),"$15,000",Sheldon Whitehouse
1,4/18/2023,ESS,Sale (Full),"$15,000",Sheldon Whitehouse
3,4/4/2023,UPS,Sale (Full),"$15,000",Shelley Moore Capito
4,4/4/2023,MCD,Sale (Partial),"$15,000",Shelley Moore Capito
5,4/4/2023,DUK,Sale (Full),"$50,000",Shelley Moore Capito
...,...,...,...,...,...
7627,1/16/2015,NOG,Purchase,"$50,000",John Hoeven
7628,1/6/2015,FCX,Sale (Full),"$100,000",John Hoeven
7640,12/12/2014,XOM,Sale (Full),"$15,000",Pat Roberts
7641,12/19/2014,BA,Purchase,"$15,000",Pat Roberts


In [106]:
# declare ticker counts variable and get the tally
tickers_counts=tickers_df['ticker'].value_counts()

# declare ticker count variable for MSFT
msft_counts=tickers_counts['MSFT']

# display the counts for MSFT
msft_counts

96

In [107]:
# declare most common ticker, get the value counts 
most_common_ticker=tickers_df['ticker'].value_counts().head(5)
most_common_ticker

ticker
AAPL    168
MSFT     96
BAC      81
DIS      79
PFE      77
Name: count, dtype: int64

In [108]:
msft_filtered_df=tickers_df[(tickers_df['ticker']=='MSFT') & (tickers_df['amount'])]
# display(msft_filtered_df)

aapl_filtered_df=tickers_df[(tickers_df['ticker']=='AAPL') & (tickers_df['amount'])]
# display(aapl_filtered_df)

bac_filtered_df=tickers_df[(tickers_df['ticker']=='BAC') & (tickers_df['amount'])]
# display(bac_filtered_df)

dis_filtered_df=tickers_df[(tickers_df['ticker']=='DIS') & (tickers_df['amount'])]
# display(dis_filtered_df)

pfe_filtered_df=tickers_df[(tickers_df['ticker']=='PFE') & (tickers_df['amount'])]
# display(pfe_filtered_df)

combined_data=pd.concat([msft_filtered_df, aapl_filtered_df, bac_filtered_df, dis_filtered_df, pfe_filtered_df], axis=0)
display(combined_data)

,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
301,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
309,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
...,...,...,...,...,...
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7326,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


In [123]:
# drop duplicates
dup_drop=combined_data.drop_duplicates().dropna()
dup_drop

ready_csv_data=dup_drop.dropna()#.set_index('transaction_date')
display(ready_csv_data)

,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
323,11/23/2022,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,11/23/2022,MSFT,Purchase,"$15,000",Tommy Tuberville
...,...,...,...,...,...
6963,9/16/2015,PFE,Purchase,"$15,000",Gary C. Peters
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


In [145]:
import yfinance as yf
from datetime import datetime, timedelta
import datetime as dt

In [146]:


# 6 years of data
six_year_end_date = datetime.today()
six_year_start_date = six_year_end_date - timedelta(days=365*9)

# # 30 days of data
# month_end_date = datetime.today()
# month_start_date = end_date - timedelta(days=30)

# # 90 days of data
# three_month_end_date = datetime.today()
# three_month_start_date = end_date - timedelta(days=90)

# # 180 days of data
# six_month_end_date = datetime.today()
# six_month_start_date = six_month_end_date - timedelta(days=180)

msft_tickers = yf.Tickers('MSFT')
msft_data=msft_tickers.tickers['MSFT'].history(start=six_year_start_date, end=six_year_end_date)
msft_ticker_df=pd.DataFrame(msft_data)
msft_ticker_df.index=msft_ticker_df.index.date
# msft_ticker_df

aapl_tickers=yf.Tickers('AAPL')
aapl_data=aapl_tickers.tickers['AAPL'].history(start=six_year_start_date, end=six_year_end_date)
aapl_ticker_df=pd.DataFrame(aapl_data)
aapl_ticker_df.index=aapl_ticker_df.index.date
# aapl_ticker_df


bac_tickers=yf.Tickers('BAC')
bac_data=bac_tickers.tickers['BAC'].history(start=six_year_start_date, end=six_year_end_date)
bac_ticker_df=pd.DataFrame(bac_data)
bac_ticker_df.index=bac_ticker_df.index.date
# bac_ticker_df

dis_tickers=yf.Tickers('DIS')
dis_data=dis_tickers.tickers['DIS'].history(start=six_year_start_date, end=six_year_end_date)
dis_ticker_df=pd.DataFrame(dis_data)
dis_ticker_df.index=dis_ticker_df.index.date
# dis_ticker_df

pfe_tickers=yf.Tickers('PFE')
pfe_data=pfe_tickers.tickers['PFE'].history(start=six_year_start_date, end=six_year_end_date)
pfe_ticker_df=pd.DataFrame(pfe_data)
pfe_ticker_df.index=pfe_ticker_df.index.date
# pfe_ticker_df

In [149]:
msft_filtered_ticker_df=msft_ticker_df[['Close']]
# msft_filtered_ticker_df

aapl_filtered_ticker_df=aapl_ticker_df[['Close']]
# display(aapl_filtered_ticker_df)

bac_filtered_ticker_df=bac_ticker_df[['Close']]
# display(bac_filtered_ticker_df)


dis_filtered_ticker_df=dis_ticker_df[['Close']]
# display(dis_filtered_ticker_df)


pfe_filtered_ticker_df=pfe_ticker_df[['Close']]
# display(pfe_filtered_ticker_df)


ready_ticker_data=pd.concat([msft_filtered_ticker_df, aapl_filtered_ticker_df, bac_filtered_ticker_df, dis_filtered_ticker_df, pfe_filtered_ticker_df], axis=1, keys=['MSFT', 'AAPL', 'BAC', 'DIS']).dropna()
display(ready_ticker_data)

,MSFT,AAPL,BAC,DIS
,Close,Close,Close,Close
2014-08-11,37.061386,21.445509,12.801787,80.527657
2014-08-12,37.335911,21.441044,12.793377,80.260773
2014-08-13,37.816341,21.724777,12.827023,80.619690
2014-08-14,37.979336,21.782867,12.885900,81.549217
2014-08-15,38.425457,21.890100,12.801787,82.165825
...,...,...,...,...
2023-08-02,327.500000,192.580002,31.190001,86.300003
2023-08-03,326.660004,191.169998,31.410000,85.489998
2023-08-04,327.779999,181.990005,31.299999,86.300003


In [150]:
msft_filtered_ticker_df=msft_ticker_df[['Close']]
# msft_filtered_ticker_df

aapl_filtered_ticker_df=aapl_ticker_df[['Close']]
# display(aapl_filtered_ticker_df)

bac_filtered_ticker_df=bac_ticker_df[['Close']]
# display(bac_filtered_ticker_df)


dis_filtered_ticker_df=dis_ticker_df[['Close']]
# display(dis_filtered_ticker_df)


pfe_filtered_ticker_df=pfe_ticker_df[['Close']]
# display(pfe_filtered_ticker_df)


ready_ticker_data=pd.concat([msft_filtered_ticker_df, aapl_filtered_ticker_df, bac_filtered_ticker_df, dis_filtered_ticker_df, pfe_filtered_ticker_df], axis=1, keys=['MSFT', 'AAPL', 'BAC', 'DIS', 'PFE']).dropna()
display(ready_ticker_data)

ready_ticker_data.columns

,MSFT,AAPL,BAC,DIS,PFE
,Close,Close,Close,Close,Close
2014-08-11,37.061386,21.445509,12.801787,80.527657,19.267242
2014-08-12,37.335911,21.441044,12.793377,80.260773,19.151297
2014-08-13,37.816341,21.724777,12.827023,80.619690,19.239965
2014-08-14,37.979336,21.782867,12.885900,81.549217,19.594612
2014-08-15,38.425457,21.890100,12.801787,82.165825,19.533234
...,...,...,...,...,...
2023-08-02,327.500000,192.580002,31.190001,86.300003,35.349998
2023-08-03,326.660004,191.169998,31.410000,85.489998,35.000000
2023-08-04,327.779999,181.990005,31.299999,86.300003,35.020000


MultiIndex([('MSFT', 'Close'),
            ('AAPL', 'Close'),
            ( 'BAC', 'Close'),
            ( 'DIS', 'Close'),
            ( 'PFE', 'Close')],
           )

In [151]:
# Create a list of new column names based on existing close columns
new_cols = [col for col, close_col in ready_ticker_data.columns]

# Assign the new column names to the columns of ready_ticker_data
ready_ticker_data.columns = new_cols

# Display the first few rows of the modified ready_ticker_data DataFrame
ready_ticker_data.head()


,MSFT,AAPL,BAC,DIS,PFE
2014-08-11,37.061386,21.445509,12.801787,80.527657,19.267242
2014-08-12,37.335911,21.441044,12.793377,80.260773,19.151297
2014-08-13,37.816341,21.724777,12.827023,80.619690,19.239965
2014-08-14,37.979336,21.782867,12.885900,81.549217,19.594612
2014-08-15,38.425457,21.890100,12.801787,82.165825,19.533234


In [152]:
ready_ticker_data.tail()

,MSFT,AAPL,BAC,DIS,PFE
2023-08-02,327.500000,192.580002,31.190001,86.300003,35.349998
2023-08-03,326.660004,191.169998,31.410000,85.489998,35.000000
2023-08-04,327.779999,181.990005,31.299999,86.300003,35.020000
2023-08-07,330.109985,178.850006,31.879999,86.830002,35.680000
2023-08-08,326.049988,179.800003,31.270000,88.129997,35.639999


In [153]:
ready_csv_data.head()

,transaction_date,ticker,type,amount,senator
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville


In [154]:
complete_df=pd.DataFrame(ready_csv_data)
complete_df

,transaction_date,ticker,type,amount,senator
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville
...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue


In [158]:
ready_csv_data['transaction_date'] = pd.to_datetime(ready_csv_data['transaction_date'])

# Convert the index of ready_ticker_data to Timestamps
ready_ticker_data.index = pd.to_datetime(ready_ticker_data.index)

duration = 30
for idx, row in ready_csv_data.iterrows():
    start = row['transaction_date']
    ticker = row['ticker']
    
    if ticker not in ready_ticker_data.columns:
        continue
        
    end = start + pd.Timedelta(days=duration)  # Use pd.Timedelta to ensure compatibility
    
    # Slice the DataFrame using loc with Timestamp objects
    df_in_timeframe = ready_ticker_data.loc[start:end, ticker]
    
    if df_in_timeframe.empty:
        continue
        
    return_in_timeframe = df_in_timeframe.iloc[-1] / df_in_timeframe.iloc[0] - 1
    ready_csv_data.loc[idx, f'{duration}_days_return'] = return_in_timeframe.item()

display(ready_csv_data)


,transaction_date,ticker,type,amount,senator,30_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746
...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018829
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188


In [159]:
# Import the necessary module
from datetime import timedelta

# Convert the 'transaction_date' column in ready_csv_data to datetime format
ready_csv_data['transaction_date'] = pd.to_datetime(ready_csv_data['transaction_date'])

# Set the duration for the time frame
duration = 60

# Loop through each row in the ready_csv_data DataFrame
for idx, row in ready_csv_data.iterrows(): 
    # Extract the transaction date and ticker from the current row
    start = row['transaction_date'].date()  # Convert to datetime.date
    ticker = row['ticker']  # AAPL
    
    # Check if the ticker is present in ready_ticker_data columns
    if ticker not in ready_ticker_data.columns:
        # If not, skip to the next row
        continue
        
    # Calculate the end date of the time frame
    end = start + timedelta(days=duration)
    
    # Extract the data for the specified ticker within the time frame
    df_in_timeframe = ready_ticker_data.loc[start:end, ticker]

    # Check if the extracted DataFrame is empty
    if df_in_timeframe.empty:
        # If empty, skip to the next row
        continue
        
    # Calculate the return within the time frame
    return_in_timeframe = df_in_timeframe.iloc[-1] / df_in_timeframe.iloc[0] - 1
    
    # Assign the calculated return to the ready_csv_data DataFrame
    ready_csv_data.loc[idx, f'{duration}_days_return'] = return_in_timeframe.item()

# Display the modified ready_csv_data DataFrame
display(ready_csv_data)


,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728
...,...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070,0.010806
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800,-0.002605
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018829,-0.002690
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188,-0.009828


In [162]:
# Convert the 'transaction_date' column in ready_csv_data to datetime format
ready_csv_data['transaction_date'] = pd.to_datetime(ready_csv_data['transaction_date'])

# Set the duration for the time frame
duration = 60

# Loop through each row in the ready_csv_data DataFrame
for idx, row in ready_csv_data.iterrows(): 
    # Extract the transaction date and ticker from the current row
    start = row['transaction_date']  # Extracted transaction date
    ticker = row['ticker']  # Extracted ticker
    
    # Check if the ticker is present in ready_ticker_data columns
    if ticker not in ready_ticker_data.columns:
        # If not, skip to the next row
        continue
        
    # Calculate the end date of the time frame
    end = start + timedelta(days=duration)
    
    # Extract the data for the specified ticker within the time frame
    df_in_timeframe = ready_ticker_data.loc[start:end, ticker]

    # Check if the extracted DataFrame is empty
    if df_in_timeframe.empty:
        # If empty, skip to the next row
        continue
        
    # Calculate the return within the time frame
    return_in_timeframe = df_in_timeframe.iloc[-1] / df_in_timeframe.iloc[0] - 1
    
    # Assign the calculated return to the ready_csv_data DataFrame
    ready_csv_data.loc[idx, f'{duration}_days_return'] = return_in_timeframe.item()

# Display the modified ready_csv_data DataFrame
display(ready_csv_data)


,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return,90_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258,0.188192
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630,0.055815
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358,-0.019971
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728,0.023115
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728,0.023115
...,...,...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070,0.010806,-0.019880
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800,-0.002605,0.039443
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018829,-0.002690,0.079199
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188,-0.009828,0.075042


In [161]:
# Convert the 'transaction_date' column in ready_csv_data to datetime format
ready_csv_data['transaction_date'] = pd.to_datetime(ready_csv_data['transaction_date'])

# Set the duration for the time frame
duration = 90

# Loop through each row in the ready_csv_data DataFrame
for idx, row in ready_csv_data.iterrows(): 
    # Extract the transaction date and ticker from the current row
    start = row['transaction_date']  # Extracted transaction date
    ticker = row['ticker']  # Extracted ticker
    
    # Check if the ticker is present in ready_ticker_data columns
    if ticker not in ready_ticker_data.columns:
        # If not, skip to the next row
        continue
        
    # Calculate the end date of the time frame
    end = start + timedelta(days=duration)
    
    # Extract the data for the specified ticker within the time frame
    df_in_timeframe = ready_ticker_data.loc[start:end, ticker]

    # Check if the extracted DataFrame is empty
    if df_in_timeframe.empty:
        # If empty, skip to the next row
        continue
        
    # Calculate the return within the time frame
    return_in_timeframe = df_in_timeframe.iloc[-1] / df_in_timeframe.iloc[0] - 1
    
    # Assign the calculated return to the ready_csv_data DataFrame
    ready_csv_data.loc[idx, f'{duration}_days_return'] = return_in_timeframe.item()

# Display the modified ready_csv_data DataFrame
display(ready_csv_data)


,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return,90_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258,0.188192
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630,0.055815
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358,-0.019971
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728,0.023115
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728,0.023115
...,...,...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070,0.010806,-0.019880
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800,-0.002605,0.039443
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018829,-0.002690,0.079199
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188,-0.009828,0.075042


In [163]:
# Convert the 'transaction_date' column in ready_csv_data to datetime format
ready_csv_data['transaction_date'] = pd.to_datetime(ready_csv_data['transaction_date'])

# Set the duration for the time frame
duration = 180

# Loop through each row in the ready_csv_data DataFrame
for idx, row in ready_csv_data.iterrows(): 
    # Extract the transaction date and ticker from the current row
    start = row['transaction_date']  # Extracted transaction date
    ticker = row['ticker']  # Extracted ticker
    
    # Check if the ticker is present in ready_ticker_data columns
    if ticker not in ready_ticker_data.columns:
        # If not, skip to the next row
        continue
        
    # Calculate the end date of the time frame
    end = start + timedelta(days=duration)
    
    # Extract the data for the specified ticker within the time frame
    df_in_timeframe = ready_ticker_data.loc[start:end, ticker]

    # Check if the extracted DataFrame is empty
    if df_in_timeframe.empty:
        # If empty, skip to the next row
        continue
        
    # Calculate the return within the time frame
    return_in_timeframe = df_in_timeframe.iloc[-1] / df_in_timeframe.iloc[0] - 1
    
    # Assign the calculated return to the ready_csv_data DataFrame
    ready_csv_data.loc[idx, f'{duration}_days_return'] = return_in_timeframe.item()

# Display the modified ready_csv_data DataFrame
display(ready_csv_data)


,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return,90_days_return,180_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258,0.188192,0.137634
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630,0.055815,0.305289
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358,-0.019971,0.310862
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
...,...,...,...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070,0.010806,-0.019880,-0.076311
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800,-0.002605,0.039443,0.012301
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018829,-0.002690,0.079199,0.056007
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188,-0.009828,0.075042,0.049532


In [165]:
display(ready_csv_data.head(10))

,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return,90_days_return,180_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258,0.188192,0.137634
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630,0.055815,0.305289
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358,-0.019971,0.310862
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
326,2022-11-22,MSFT,Sale (Partial),"$250,000",Tommy Tuberville,-0.027915,-0.019630,0.055815,0.305289
359,2022-10-28,MSFT,Sale (Full),"$15,000",Tommy Tuberville,0.052221,0.007452,0.054390,0.258932
363,2022-10-24,MSFT,Purchase,"$50,000",Tommy Tuberville,0.004157,-0.031738,-0.025695,0.161913
365,2022-10-13,MSFT,Purchase,"$500,000",Tommy Tuberville,0.054944,0.081035,0.009368,0.213872
402,2022-10-05,MSFT,Sale (Partial),"$250,000",Tommy Tuberville,-0.111597,0.026239,-0.035894,0.158752


In [166]:
# plot bar with 30 day return
ready_csv_data.hvplot.bar(x='transaction_date', y='30_days_return')

:Bars   [transaction_date]   (30_days_return)

In [167]:
ready_ticker_data.to_csv('ready_ticker_data.csv', index=False)